In [1]:
import numpy as np
import cv2

In [2]:
from computeH import computeH
from utils import to_homogenous_coords

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
im1 = cv2.imread('crop1.jpg')
im2 = cv2.imread('crop2.jpg')

h1, w1, _ = im1.shape
h2, w2, _ = im2.shape

scale = max(h1, w1, h2, w2) / 2
t1_scale = max(h1, w1) / 2
t2_scale = max(h2, w2) / 2

In [7]:
#t1, t2 = get_correspondences(im1, im2)

t1 = np.load('cc1.npy').T
t2 = np.load('cc2.npy').T

# t1 = t1.T
# t2 = t2.T

In [8]:
# t1_scaled = t1 / np.array([[w1/2], [h1/2]])
# t2_scaled = t2 / np.array([[w2/2], [h2/2]])

# t1_scaled = t1 / scale
# t2_scaled = t2 / scale

t1_scaled = t1 / t1_scale
t2_scaled = t2 / t2_scale

In [16]:
H = computeH(t1_scaled, t2_scaled)

In [10]:
t1_hom = to_homogenous_coords(t1_scaled)

In [11]:
p_prime = np.matmul(H, t1_hom)

In [12]:
((p_prime / p_prime[2]) * t2_scale)[1]

array([139.01423494, 178.11736161, 216.22879774, 233.73285354,
       227.31363183, 240.04683989, 290.85364417, 405.06945867,
       384.10577465, 234.87070931, 230.00661652, 111.10793294])

In [13]:
t2[1]

array([139.21355932, 177.68813559, 216.12542373, 234.43728814,
       227.41355932, 239.36949153, 290.8779661 , 405.4       ,
       384.        , 234.4       , 230.2       , 111.34915254])

In [14]:
t2_prime = (p_prime[:2] / p_prime[2]) * t2_scale

In [15]:
np.sqrt(np.mean(np.square(t2_prime - t2)))

0.6352619713357548

In [25]:
def warpImage(inputIm, refIm, H):
    pass

In [26]:
im1

array([[[ 23, 148, 132],
        [ 40, 165, 149],
        [ 58, 183, 164],
        ...,
        [ 90,  49,  26],
        [195, 149, 125],
        [105,  56,  30]],

       [[ 36, 160, 144],
        [ 40, 165, 146],
        [ 51, 173, 155],
        ...,
        [ 64,  24,   0],
        [172, 126,  95],
        [207, 160, 128]],

       [[ 78, 200, 182],
        [ 67, 190, 170],
        [ 59, 180, 160],
        ...,
        [133,  94,  56],
        [130,  86,  45],
        [170, 125,  82]],

       ...,

       [[121, 192, 226],
        [119, 191, 225],
        [122, 196, 230],
        ...,
        [152, 217, 248],
        [141, 207, 236],
        [123, 189, 218]],

       [[123, 194, 228],
        [124, 196, 230],
        [127, 199, 233],
        ...,
        [131, 192, 224],
        [142, 202, 232],
        [142, 202, 232]],

       [[117, 188, 222],
        [125, 197, 231],
        [127, 199, 233],
        ...,
        [155, 214, 246],
        [143, 200, 231],
        [126, 183, 214]]

In [27]:
t1_scale

240.0

In [28]:
t2_scale

225.0

In [29]:
new_im = np.zeros((1000, 1000))

In [31]:
im2.shape

(450, 317, 3)

In [45]:
for i in range(points.shape[1]):
    

(2, 142650)

In [48]:
H_inv = np.linalg.inv(H)

In [53]:
p = np.array([(i, j) for i in range(450) for j in range(317)]).T

In [54]:
p_hom = to_homogenous_coords(p)

In [56]:
p_prime = np.matmul(H_inv, p_hom)